In [1]:
import cv2
import mediapipe as mp
import math
import pandas as pd
import os

In [4]:
mp_pose = mp.solutions.pose

pose = mp_pose.Pose(static_image_mode = True)

def calc_dist_3d(p1,p2):
    distance = math.sqrt((p1.x-p2.x)**2+(p1.y-p2.y)**2+(p1.z-p2.z)**2)
    return distance

print("It's gonna be Allright!")

It's gonna be Allright!


In [29]:
def calc_femur_and_torso(img_path):
    img = cv2.imread(img_path)

    femur_length = 0
    torso_length = 0
    tfr_ratio = 0
    L_is_reliable = False
    R_is_reliable = False

    if img is None:
        print(f"'{img_path}' is not detected. plz check the file name")

    else:
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        results = pose.process(img_rgb)

        if results.pose_landmarks:
            print("ALLRIGHT!!!")
            landmarks = results.pose_landmarks.landmark

            #For Left
            L_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
            L_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
            L_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value]

            L_shoulder_vis = L_shoulder.visibility
            L_hip_vis = L_hip.visibility
            L_knee_vis = L_knee.visibility

            L_vis_avg = L_shoulder_vis+L_hip_vis+L_knee_vis/3.0

            #For Left
            R_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
            R_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
            R_knee = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value]

            R_shoulder_vis = R_shoulder.visibility
            R_hip_vis = R_hip.visibility
            R_knee_vis = R_knee.visibility

            R_vis_avg = L_shoulder_vis+L_hip_vis+L_knee_vis/3.0

            vis_threshold = 0.8
            R_is_vis = False
            L_is_vis = False
            select_L = False
            select_R = False

            if(L_shoulder_vis>vis_threshold and
            L_hip_vis>vis_threshold and
            L_knee_vis>vis_threshold):
                L_is_vis = True

            if(R_shoulder_vis>vis_threshold and
            R_hip_vis>vis_threshold and
            R_knee_vis>vis_threshold):
                R_is_vis = True

            if(L_is_vis or R_is_vis):
                if(L_is_vis and R_is_vis):
                    if(L_vis_avg>R_vis_avg):
                        select_L = True
                    else:
                        select_R = True
            
                elif((L_is_vis and R_is_vis != True) or select_L):
                    #The Key Value!
                    #femur length = hip ~ knee
                    femur_length = calc_dist_3d(L_hip,L_knee)

                    #torso length = shoulder ~ hip
                    torso_length = calc_dist_3d(L_shoulder, L_hip)
                    print("Left things are selected.")

                elif((R_is_vis and L_is_vis != True) or select_R):
                    #The Key Value!
                    #femur length = hip ~ knee
                    femur_length = calc_dist_3d(R_hip,R_knee)

                    #torso length = shoulder ~ hip
                    torso_length = calc_dist_3d(R_shoulder, R_hip)
                    print("Right things are selected.")

                if torso_length >0:
                    tfr_ratio = femur_length/torso_length

                print(f"The estimated relative femur length : {femur_length:.4f}")
                print(f"The estimated relative torso length : {torso_length:.4f}")
                print(f"The estimated TFR(femur/torso) ratio : {tfr_ratio:.4f}")


            else:
                print(f"\n [WARNING] TOO LOW RELIABILITY OF THE POINTS (criteria : {vis_threshold}).")
                print("Plz check the angle of the photo or take another snap")

        else:
            print("Cannot detect the points on the image.")
            print("Plz check whether the image is for standing pose.")

In [30]:
calc_femur_and_torso('allright1.png')
print()
calc_femur_and_torso('allright2.png')
print()
calc_femur_and_torso('cbum1.png')

ALLRIGHT!!!
Right things are selected.
The estimated relative femur length : 0.2228
The estimated relative torso length : 0.2586
The estimated TFR(femur/torso) ratio : 0.8617

Cannot detect the points on the image.
Plz check whether the image is for standing pose.

ALLRIGHT!!!
Left things are selected.
The estimated relative femur length : 0.2193
The estimated relative torso length : 0.3106
The estimated TFR(femur/torso) ratio : 0.7062
